In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
from preprocessing import load_if_in_country_or_lang
from preprocessing import categorize_views_for_df
from processing import filter_by_region_cat_code, build_regression_dataframe, load_preprocessed_events

In [5]:
df_crawled = load_preprocessed_events()
df_crawled = load_if_in_country_or_lang(df_crawled)
df_crawled = categorize_views_for_df(df_crawled)
df_crawled.head()

,event_id,pagetitle,event_date,economic_region,country,continent,cat,categories,views_7_sum,views_before_mean,...,Country/Area,Year,GDP_pc,GDP,oecd,date_month,views_baseline,in_code_lang,in_code_region,views_baseline_cat
0,Q100021,Pakistan International Airlines Flight 661,2016-12-07,Global South,Pakistan,Asia,disaster,"['Category:2016 disasters in Pakistan', 'Categ...",248274,0.000000,...,Pakistan,2016,1362.860008,2.775210e+11,False,2016-12-01,8.684000e+07,True,False,>10^7
1,Q100231328,Gretchen Whitmer kidnapping plot,2020-10-08,Global North,United States,North America,disaster,"['Category:2020 crimes in the United States', ...",74144,0.000000,...,United States,2020,63122.594145,2.089375e+13,True,2020-10-01,3.137752e+09,True,True,>10^9
2,Q100256872,Killing of Babu Lal Vaishnav,2020-10-08,Global South,India,Asia,disaster,"['Category:2020 crimes in India', 'Category:20...",427,0.000000,...,India,2020,1930.971218,2.664749e+12,False,2020-10-01,7.142770e+08,True,False,>10^8
3,Q100279235,Miss Vietnam 2020,2020-11-20,Global South,Vietnam,Asia,culture,"['Category:2020 beauty pageants', 'Category:Be...",1444,49.285714,...,Vietnam,2020,2785.724225,2.711584e+11,False,2020-11-01,1.619000e+07,False,False,>10^7
4,Q100288161,2020 Ganja missile attacks,2020-10-11,Global South,Azerbaijan,Asia,disaster,"['Category:2020 in Azerbaijan', 'Category:Arti...",7410,0.000000,...,Azerbaijan,2020,4202.232239,4.260718e+10,False,2020-10-01,3.088000e+06,False,False,>10^6


In [6]:
df_pageviews_all = pd.read_csv(f'countries/pageviews_countries.csv')
df_pageviews_all

# Step 1: Poisson

In [77]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [78]:
df_filtered dir
data_filtered = build_regression_dataframe(df_filtered)

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [79]:
lang_dfs = {}
for lang in ['de', 'es', 'en', 'it']:
    lang_dfs[lang] = df_filtered[df_filtered.code == lang]

In [140]:
df_rel = lang_dfs['en'].copy()
glm_p = smf.glm(formula='views_7_sum ~ cat * gni_region', data=df_rel, family=sm.families.Poisson()).fit()
mu = glm_p.mu
glm_p.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            views_7_sum   No. Observations:                 7606
Model:                            GLM   Df Residuals:                     7578
Model Family:                 Poisson   Df Model:                           27
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -5.2559e+08
Date:                Mon, 04 Apr 2022   Deviance:                   1.0511e+09
Time:                        10:26:38   Pearson chi2:                 5.59e+09
No. Iterations:                    11                                         
Covariance Type:            nonrobust                                         
============================================================================================================================
                                                               coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------
Intercept                                                    9.5188      0.001   1.27e+04      0.000       9.517       9.520
cat[T.disaster]                                              1.2098      0.001   1387.758      0.000       1.208       1.211
cat[T.politics]                                              0.6932      0.001    834.445      0.000       0.692       0.695
cat[T.sports]                                                0.8418      0.001   1074.164      0.000       0.840       0.843
gni_region[T.Europe & Central Asia]                          0.8188      0.001    964.228      0.000       0.817       0.820
gni_region[T.Latin America & Caribbean]                     -1.7544      0.005   -366.553      0.000      -1.764      -1.745
gni_region[T.Middle East & North Africa]                    -1.2333      0.016    -77.580      0.000      -1.264      -1.202
gni_region[T.North America]                                  1.7566      0.001   2245.955      0.000       1.755       1.758
gni_region[T.South Asia]                                     0.2310      0.002    149.660      0.000       0.228       0.234
gni_region[T.Sub-Saharan Africa]                            -1.3525      0.005   -296.183      0.000      -1.361      -1.344
cat[T.disaster]:gni_region[T.Europe & Central Asia]         -0.0614      0.001    -62.412      0.000      -0.063      -0.059
cat[T.politics]:gni_region[T.Europe & Central Asia]         -0.4955      0.001   -526.670      0.000      -0.497      -0.494
cat[T.sports]:gni_region[T.Europe & Central Asia]           -1.2111      0.001  -1355.310      0.000      -1.213      -1.209
cat[T.disaster]:gni_region[T.Latin America & Caribbean]      1.8084      0.005    373.275      0.000       1.799       1.818
cat[T.politics]:gni_region[T.Latin America & Caribbean]      1.8311      0.005    378.445      0.000       1.822       1.841
cat[T.sports]:gni_region[T.Latin America & Caribbean]        1.9122      0.005    398.645      0.000       1.903       1.922
cat[T.disaster]:gni_region[T.Middle East & North Africa]     1.8870      0.016    118.629      0.000       1.856       1.918
cat[T.politics]:gni_region[T.Middle East & North Africa]     1.3927      0.016     87.477      0.000       1.361       1.424
cat[T.sports]:gni_region[T.Middle East & North Africa]       1.3446      0.016     84.527      0.000       1.313       1.376
cat[T.disaster]:gni_region[T.North America]                 -0.6184      0.001   -669.284      0.000      -0.620      -0.617
cat[T.politics]:gni_region[T.North America]                 -1.0657      0.001  -1217.735      0.000      -1.067      -1.064
cat[T.sports]:gni_region[T.North America]                   -1.3967      0.001  -16

## Step 2: Aux OLS for estimating alpha

In [41]:
df_rel['mu'] = mu
df_rel['aux_dep'] = df_rel.apply(lambda x: (x['views_7_sum'] - x['mu'])**2 - x['mu'], axis=1)

In [48]:
ols_aux = smf.ols('aux_dep ~ mu - 1', data=df_rel).fit()
alpha_comp = ols_aux.params['mu']
ols_aux.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                aux_dep   R-squared (uncentered):                   0.003
Model:                            OLS   Adj. R-squared (uncentered):              0.003
Method:                 Least Squares   F-statistic:                              20.63
Date:                Fri, 01 Apr 2022   Prob (F-statistic):                    5.65e-06
Time:                        12:35:41   Log-Likelihood:                     -2.1996e+05
No. Observations:                7606   AIC:                                  4.399e+05
Df Residuals:                    7605   BIC:                                  4.399e+05
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
mu          9.718e+05   2.14e+05      4.542      0.000    5.52e+05    1.39e+06
==============================================================================
Omnibus:                    24550.376   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       3457276495.096
Skew:                          53.368   Prob(JB):                         0.00
Kurtosis:                    3304.168   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [61]:
ols_aux.pvalues

mu    0.000006
dtype: float64

In [51]:
## Step 3: Fit NB 
nb_fit_alpha = smf.glm(formula='views_7_sum ~ gdp_pc * cat * gni_region', data=df_rel, family = sm.families.NegativeBinomial(alpha=alpha_comp)).fit()
nb_fit_alpha.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            views_7_sum   No. Observations:                 7606
Model:                            GLM   Df Residuals:                     7578
Model Family:        NegativeBinomial   Df Model:                           27
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.6935e+05
Date:                Fri, 01 Apr 2022   Deviance:                     0.031310
Time:                        12:36:39   Pearson chi2:                    0.164
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
============================================================================================================================
                                                               coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------
Intercept                                                    9.5188     86.128      0.111      0.912    -159.289     178.326
cat[T.disaster]                                              1.2098    127.484      0.009      0.992    -248.654     251.074
cat[T.politics]                                              0.6932    104.114      0.007      0.995    -203.366     204.753
cat[T.sports]                                                0.8418     95.167      0.009      0.993    -185.682     187.365
gni_region[T.Europe & Central Asia]                          0.8188    110.583      0.007      0.994    -215.921     217.558
gni_region[T.Latin America & Caribbean]                     -1.7544    241.998     -0.007      0.994    -476.062     472.554
gni_region[T.Middle East & North Africa]                    -1.2333    989.533     -0.001      0.999   -1940.683    1938.217
gni_region[T.North America]                                  1.7566    106.399      0.017      0.987    -206.781     210.294
gni_region[T.South Asia]                                     0.2310    194.385      0.001      0.999    -380.756     381.218
gni_region[T.Sub-Saharan Africa]                            -1.3525    277.181     -0.005      0.996    -544.617     541.912
cat[T.disaster]:gni_region[T.Europe & Central Asia]         -0.0614    159.688     -0.000      1.000    -313.043     312.920
cat[T.politics]:gni_region[T.Europe & Central Asia]         -0.4955    130.054     -0.004      0.997    -255.397     254.406
cat[T.sports]:gni_region[T.Europe & Central Asia]           -1.2111    119.858     -0.010      0.992    -236.129     233.707
cat[T.disaster]:gni_region[T.Latin America & Caribbean]      1.8084    290.596      0.006      0.995    -567.748     571.365
cat[T.politics]:gni_region[T.Latin America & Caribbean]      1.8311    269.562      0.007      0.995    -526.500     530.162
cat[T.sports]:gni_region[T.Latin America & Caribbean]        1.9122    248.781      0.008      0.994    -485.689     489.514
cat[T.disaster]:gni_region[T.Middle East & North Africa]     1.8870    998.913      0.002      0.998   -1955.946    1959.720
cat[T.politics]:gni_region[T.Middle East & North Africa]     1.3927   1001.214      0.001      0.999   -1960.951    1963.737
cat[T.sports]:gni_region[T.Middle East & North Africa]       1.3446    995.355      0.001      0.999   -1949.515    1952.204
cat[T.disaster]:gni_region[T.North America]                 -0.6184    161.494     -0.004      0.997    -317.140     315.903
cat[T.politics]:gni_region[T.North America]                 -1.0657    126.710     -0.008      0.993    -249.413     247.282
cat[T.sports]:gni_region[T.North America]                   -1.3967    119.049     

In [53]:
nb_fit_alpha16 = smf.glm(formula='views_7_sum ~ cat * gni_region', data=df_rel, family = sm.families.NegativeBinomial(alpha=16)).fit()
nb_fit_alpha16.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            views_7_sum   No. Observations:                 7606
Model:                            GLM   Df Residuals:                     7578
Model Family:        NegativeBinomial   Df Model:                           27
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -88069.
Date:                Fri, 01 Apr 2022   Deviance:                       1901.6
Time:                        12:36:54   Pearson chi2:                 9.94e+03
No. Iterations:                     9                                         
Covariance Type:            nonrobust                                         
============================================================================================================================
                                                               coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------
Intercept                                                    9.5188      0.349     27.237      0.000       8.834      10.204
cat[T.disaster]                                              1.2098      0.517      2.339      0.019       0.196       2.224
cat[T.politics]                                              0.6932      0.422      1.641      0.101      -0.135       1.521
cat[T.sports]                                                0.8418      0.386      2.180      0.029       0.085       1.599
gni_region[T.Europe & Central Asia]                          0.8188      0.449      1.825      0.068      -0.061       1.698
gni_region[T.Latin America & Caribbean]                     -1.7544      0.982     -1.787      0.074      -3.679       0.170
gni_region[T.Middle East & North Africa]                    -1.2333      4.015     -0.307      0.759      -9.103       6.637
gni_region[T.North America]                                  1.7566      0.432      4.069      0.000       0.910       2.603
gni_region[T.South Asia]                                     0.2310      0.789      0.293      0.770      -1.315       1.777
gni_region[T.Sub-Saharan Africa]                            -1.3525      1.125     -1.202      0.229      -3.557       0.852
cat[T.disaster]:gni_region[T.Europe & Central Asia]         -0.0614      0.648     -0.095      0.925      -1.331       1.209
cat[T.politics]:gni_region[T.Europe & Central Asia]         -0.4955      0.528     -0.939      0.348      -1.530       0.539
cat[T.sports]:gni_region[T.Europe & Central Asia]           -1.2111      0.486     -2.490      0.013      -2.164      -0.258
cat[T.disaster]:gni_region[T.Latin America & Caribbean]      1.8084      1.179      1.534      0.125      -0.503       4.120
cat[T.politics]:gni_region[T.Latin America & Caribbean]      1.8311      1.094      1.674      0.094      -0.313       3.975
cat[T.sports]:gni_region[T.Latin America & Caribbean]        1.9122      1.009      1.894      0.058      -0.066       3.891
cat[T.disaster]:gni_region[T.Middle East & North Africa]     1.8870      4.053      0.466      0.642      -6.057       9.831
cat[T.politics]:gni_region[T.Middle East & North Africa]     1.3927      4.063      0.343      0.732      -6.570       9.355
cat[T.sports]:gni_region[T.Middle East & North Africa]       1.3446      4.039      0.333      0.739      -6.571       9.261
cat[T.disaster]:gni_region[T.North America]                 -0.6184      0.655     -0.944      0.345      -1.903       0.666
cat[T.politics]:gni_region[T.North America]                 -1.0657      0.514     -2.073      0.038      -2.073      -0.058
cat[T.sports]:gni_region[T.North America]                   -1.3967      0.483     

# Combined Code in single function (example for english and GDP)

In [91]:
df_rel['population']

0         203631356
1         329484123
2        1380004385
3          97338583
4          10093121
            ...    
14911      66460344
14912      67215293
14913      67215293
14914        105697
14915        105697
Name: population, Length: 7606, dtype: int64

In [209]:
df_rel.columns

Index(['event_id', 'pagetitle', 'event_date', 'economic_region', 'country',
       'continent', 'cat', 'categories', 'views_7_sum', 'views_before_mean',
       'views_before_sum', 'views_before_max', 'views_before_min',
       'views_before_median', 'views_before_increase', 'list_views_7_days',
       'diff_days', 'year', 'planed', 'surprising', 'factor', 'page_creation',
       'edits_7_sum', 'edits_before_min', 'edits_before_mean',
       'edits_before_max', 'edits_before_median', 'edits_before_sum', 'code',
       'bing_hits', 'gni_class', 'gni_region', 'population', 'Country/Area',
       'Year', 'GDP_pc', 'GDP', 'oecd', 'date_month', 'views_baseline',
       'in_code_lang', 'in_code_region', 'views_baseline_cat', 'gdp_z',
       'pop_z'],
      dtype='object')

In [215]:
from regression import fit_nb_with_estimated_alpha, estimate_alpha
df_rel = df_filtered[df_filtered.code=='en'].copy()
#df_rel['views_7_sum'] = np.log1p(df_rel['views_7_sum'])
df_rel['gdp_z'] = (df_rel['GDP_pc'] - np.mean(df_rel['GDP_pc'])) / np.std(df_rel['GDP_pc'], ddof=1)
df_rel['pop_z'] = (df_rel['population'] - np.mean(df_rel['population'])) / np.std(df_rel['population'], ddof=1)
nb_fit = fit_nb_with_estimated_alpha(df_rel, 'views_7_sum ~ (gdp_z * cat)', est_method='lbfgs')
nb_fit.summary()

Estimated alpha as 16.316754362990473 (p=0.0005150813042955219)
Deviance: 1906.573873748549|Null-deviance: 1987.7841889442748|Chi-sq (Table): 7801.895135809606


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            views_7_sum   No. Observations:                 7606
Model:                            GLM   Df Residuals:                     7598
Model Family:        NegativeBinomial   Df Model:                            7
Link Function:                    log   Scale:                          1.0000
Method:                         lbfgs   Log-Likelihood:                -88200.
Date:                Tue, 05 Apr 2022   Deviance:                       1906.6
Time:                        07:08:49   Pearson chi2:                 8.56e+03
No. Iterations:                    14                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                10.3611      0.165     62.980      0.000      10.039      10.684
cat[T.disaster]           1.0913      0.226      4.830      0.000       0.649       1.534
cat[T.politics]           0.2368      0.187      1.263      0.207      -0.131       0.604
cat[T.sports]            -0.0414      0.176     -0.235      0.814      -0.387       0.304
gdp_z                     0.6446      0.166      3.894      0.000       0.320       0.969
gdp_z:cat[T.disaster]    -0.2797      0.217     -1.286      0.198      -0.706       0.147
gdp_z:cat[T.politics]    -0.4410      0.188     -2.342      0.019      -0.810      -0.072
gdp_z:cat[T.sports]      -0.6238      0.178     -3.514      0.000      -0.972      -0.276
=========================================================================================
"""

In [200]:
from regression import fit_nb_with_estimated_alpha, estimate_alpha
df_rel = df_filtered[df_filtered.code=='en'].copy()
#df_rel['views_7_sum'] = np.log1p(df_rel['views_7_sum'])
df_rel['gdp_z'] = (df_rel['GDP_pc'] - np.mean(df_rel['GDP_pc'])) / np.std(df_rel['GDP_pc'], ddof=1)
df_rel['pop_z'] = (df_rel['population'] - np.mean(df_rel['population'])) / np.std(df_rel['population'], ddof=1)
nb_fit = fit_nb_with_estimated_alpha(df_rel, 'views_7_sum ~ (gdp_z * cat)', offset=df_rel.pop_z.values)
nb_fit.summary()

Estimated alpha as 16.316700621514215 (p=0.0005150769391426911)
Deviance: 781.7787835052168|Null-deviance: 678.974655554053|Chi-sq (Table): 2753.488606123985


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            views_7_sum   No. Observations:                 2641
Model:                            GLM   Df Residuals:                     2633
Model Family:        NegativeBinomial   Df Model:                            7
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -31701.
Date:                Mon, 04 Apr 2022   Deviance:                       781.78
Time:                        11:27:38   Pearson chi2:                 3.49e+03
No. Iterations:                    12                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                10.8339      0.290     37.383      0.000      10.266      11.402
cat[T.disaster]           1.7367      0.379      4.577      0.000       0.993       2.480
cat[T.politics]           0.8176      0.355      2.301      0.021       0.121       1.514
cat[T.sports]             1.5319      0.311      4.929      0.000       0.923       2.141
gdp_z                     1.1932      0.252      4.732      0.000       0.699       1.687
gdp_z:cat[T.disaster]    -1.1153      0.323     -3.453      0.001      -1.748      -0.482
gdp_z:cat[T.politics]    -1.1996      0.310     -3.871      0.000      -1.807      -0.592
gdp_z:cat[T.sports]      -1.9238      0.271     -7.104      0.000      -2.455      -1.393
=========================================================================================
"""

In [178]:
nb_fit = fit_nb_with_estimated_alpha(df_rel, 'views_7_sum ~ (gni_class * cat)', est_method='lbfgs')
nb_fit.summary()

Estimated alpha as 16.830884942041305 (p=0.0005796363417679687)
Deviance: 1854.6305493021741| Null-deviance: 1927.064802899287|Chi-sq (Table): 7793.788358081236


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            views_7_sum   No. Observations:                 7606
Model:                            GLM   Df Residuals:                     7590
Model Family:        NegativeBinomial   Df Model:                           15
Link Function:                    log   Scale:                          1.0000
Method:                         lbfgs   Log-Likelihood:                -88377.
Date:                Mon, 04 Apr 2022   Deviance:                       1854.6
Time:                        11:08:11   Pearson chi2:                 8.18e+03
No. Iterations:                    34                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                          10.7978      0.177     60.900      0.000      10.450      11.145
gni_class[T.L]                     -3.8605      5.627     -0.686      0.493     -14.888       7.167
gni_class[T.LM]                    -1.2605      0.554     -2.277      0.023      -2.345      -0.176
gni_class[T.UM]                    -0.6718      0.603     -1.114      0.265      -1.854       0.511
cat[T.disaster]                     0.8621      0.274      3.143      0.002       0.325       1.400
cat[T.politics]                    -0.0768      0.206     -0.373      0.709      -0.480       0.327
cat[T.sports]                      -0.5611      0.193     -2.907      0.004      -0.939      -0.183
gni_class[T.L]:cat[T.disaster]      3.1140      5.647      0.551      0.581      -7.955      14.183
gni_class[T.LM]:cat[T.disaster]     0.5345      0.670      0.797      0.425      -0.779       1.848
gni_class[T.UM]:cat[T.disaster]     0.0890      0.714      0.125      0.901      -1.310       1.488
gni_class[T.L]:cat[T.politics]      3.4444      5.658      0.609      0.543      -7.646      14.535
gni_class[T.LM]:cat[T.politics]     1.0194      0.634      1.609      0.108      -0.223       2.261
gni_class[T.UM]:cat[T.politics]     0.2765      0.661      0.418      0.676      -1.020       1.573
gni_class[T.L]:cat[T.sports]        3.3736      5.791      0.583      0.560      -7.977      14.725
gni_class[T.LM]:cat[T.sports]       1.4249      0.628      2.269      0.023       0.194       2.656
gni_class[T.UM]:cat[T.sports]       0.9541      0.622      1.534      0.125      -0.265       2.173
===================================================================================================
"""

In [175]:
nb_fit = fit_nb_with_estimated_alpha(df_rel, 'views_7_sum ~1', est_method='lbfgs')
nb_fit.summary()

Estimated alpha as 21.104540887830716 (p=0.0006172530771340359)
Deviance: 1536.8413999051863| Null-deviance: 1536.841399914345|Chi-sq (Table): 7808.98852021539
9.158611646853387e-09 nan


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            views_7_sum   No. Observations:                 7606
Model:                            GLM   Df Residuals:                     7605
Model Family:        NegativeBinomial   Df Model:                            0
Link Function:                    log   Scale:                          1.0000
Method:                         lbfgs   Log-Likelihood:                -89715.
Date:                Mon, 04 Apr 2022   Deviance:                       1536.8
Time:                        10:58:25   Pearson chi2:                 7.61e+03
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.6107      0.053    201.433      0.000      10.507      10.714
==============================================================================
"""

In [174]:
nb_fit?

Type:            GLMResultsWrapper
String form:     <statsmodels.genmod.generalized_linear_model.GLMResultsWrapper object at 0x7fb336a1a290>
File:            /opt/conda/lib/python3.7/site-packages/statsmodels/genmod/generalized_linear_model.py
Docstring:      
Class to contain GLM results.

GLMResults inherits from statsmodels.LikelihoodModelResults

Attributes
----------
df_model : float
    See GLM.df_model
df_resid : float
    See GLM.df_resid
fit_history : dict
    Contains information about the iterations. Its keys are `iterations`,
    `deviance` and `params`.
model : class instance
    Pointer to GLM model instance that called fit.
nobs : float
    The number of observations n.
normalized_cov_params : ndarray
    See GLM docstring
params : ndarray
    The coefficients of the fitted model.  Note that interpretation
    of the coefficients often depends on the distribution family and the
    data.
pvalues : ndarray
    The two-tailed p-values for the parameters.
scale : float
    

In [146]:
nb_fit.df_resid

7590

In [166]:
from scipy.stats import chi2

In [169]:
chi2.ppf(1-0.05, df=7590)

7793.788358081236

In [137]:
estimate_alpha??

Signature: estimate_alpha(df_regression, formula, est_method='IRLS')
Docstring: <no docstring>
Source:   
def estimate_alpha(df_regression, formula, est_method='IRLS'):
    df_rel = df_regression.copy()
    glm_p = smf.glm(formula=formula, data=df_rel, family=sm.families.Poisson()).fit(est_method=est_method)
    print(est_method, glm_p.method)
    df_rel['mu'] = glm_p.mu
    df_rel['aux_dep'] = df_rel.apply(lambda x: ((x['views_7_sum'] - x['mu']) ** 2 - x['mu']) / x['mu'], axis=1)
    ols_aux = smf.ols('aux_dep ~ mu - 1', data=df_rel).fit()
    # print(ols_aux.summary())
    return glm_p, ols_aux.params['mu'], ols_aux.pvalues['mu']
File:      ~/work/steinkasserer/regression.py
Type:      function


# Code Denis:

In [ ]:

df = pd.read_csv('data/ARM_data/wiki/data_filtered.csv')
print(df.head())
print(df.describe())

# df = df[df['code'] == 'en']

# intepretation: increase by one std in gdp per capita
df['gdp_z'] = (df['GDP_pc'] - np.mean(df['GDP_pc'])) / np.std(df['GDP_pc'], ddof=1)

print(df.head())
print(df.describe())

poisson = smf.glm(formula = 'views_7_sum ~ gdp_z * cat * code', data=df, family=sm.families.Poisson())
results = poisson.fit()
print(results.summary())
print('deviance:', results.deviance)
print('null deviance:', results.null_deviance)

# compute overdispersion factor (we would need to correct the standrad errors by multiplying with the square root of this factor)
sum_z_squared = np.sum(results.resid_pearson ** 2)    
degrees = len(df) - results.df_model - 1
overdispersion =  sum_z_squared / degrees
print('Overdispersion factor: ', overdispersion)
print('p-value of the observations: ', st.chi2.sf(sum_z_squared, degrees))

# fit auxillary ols to estimate alpha for negative binomial
# alternatively: simply increase alpha in neg. bin. until we reduce overdispersion to approx. 1
y = ((df.views_7_sum - results.mu) ** 2 - results.mu) / results.mu
df_aux = pd.DataFrame({'y': y, 'mu': results.mu})
lr = smf.ols(formula = 'y ~ mu - 1', data=df_aux)
results = lr.fit()
print(results.summary())
alpha = results.params['mu']
print('alpha:', alpha)

# fit negative binomial (alpha=16 works good for all languages)
nb = smf.glm(formula = 'views_7_sum ~ gdp_z * cat * gni_cat', data=df, family=sm.families.NegativeBinomial(alpha=alpha))
results = nb.fit(method="lbfgs")
print(results.summary())
print('deviance:', results.deviance)
print('null deviance:', results.null_deviance)

print('chi squared critical value for a good fit:', st.chi2.ppf(0.95, results.df_resid))
print('p-value for a bad fit:', st.chi2.sf(results.deviance, results.df_resid))

# compute overdispersion factor (if this is around 1 then we are good to go)
sum_z_squared = np.sum(results.resid_pearson ** 2)    
degrees = len(df) - results.df_model - 1
overdispersion =  sum_z_squared / degrees
print('Overdispersion factor: ', overdispersion)
print('p-value of the observations: ', st.chi2.sf(sum_z_squared, degrees))
